In [1]:
from __future__ import annotations

from vtkbone import vtkboneAIMReader

from bonelab.util.aim_calibration_header import get_aim_density_equation
from bonelab.util.vtk_util import vtkImageData_to_numpy
import numpy as np
from enum import Enum

import SimpleITK as sitk
import pyvista as pv
import vtk

import os

In [4]:
sample_visualizations_dir = os.path.join(
    "/", "Users", "nathanneeteson", "Documents", "Data",
    "Images", "AutoKneeAnalysis", "overview_figure"
)

image_name = "SLTC007L"

radius_dir = os.path.join(
    "/", "Users", "nathanneeteson", "Documents", "Data",
    "Images", "HIPFX"    
)

radius_name = "HIPFXF_0001_RR"

In [3]:
resample_factor = 2

reader = vtkboneAIMReader()
reader.DataOnCellsOff()

image_data = {}
image_data["F"] = {}
image_data["T"] = {}

for bone in ["F", "T"]:
    for image_type in ["CORT", "TRAB", "ROI_ALL"]:
        
        reader.SetFileName(os.path.join(
            sample_visualizations_dir,
            f"{image_name}_{bone}_{image_type}.AIM"
        ))
        reader.Update()
        
        image_data[bone][image_type] = pv.wrap(reader.GetOutput())
        
        
        image_data[bone][image_type] = pv.create_grid(
            image_data[bone][image_type],
            dimensions=[d//resample_factor for d in image_data[bone][image_type].dimensions]
        ).sample(image_data[bone][image_type], categorical=True, progress_bar=True)
        
       

Resampling array Data from a Passed Mesh onto Mesh: 100%|██████████████████████████████████████████████████████████████████████████████████████[00:43<00:00]
Resampling array Data from a Passed Mesh onto Mesh: 100%|██████████████████████████████████████████████████████████████████████████████████████[00:45<00:00]
Resampling array Data from a Passed Mesh onto Mesh: 100%|██████████████████████████████████████████████████████████████████████████████████████[00:48<00:00]
Resampling array Data from a Passed Mesh onto Mesh: 100%|██████████████████████████████████████████████████████████████████████████████████████[00:18<00:00]
Resampling array Data from a Passed Mesh onto Mesh: 100%|██████████████████████████████████████████████████████████████████████████████████████[00:17<00:00]
Resampling array Data from a Passed Mesh onto Mesh: 100%|██████████████████████████████████████████████████████████████████████████████████████[00:21<00:00]


In [27]:
image_data["R"] = {}

for image_type in ["CORT", "TRAB"]:
    reader.SetFileName(os.path.join(
        radius_dir,
        f"{radius_name}_{image_type}_MASK.AIM"
    ))
    reader.Update()
    
    image_data["R"][image_type] = pv.wrap(reader.GetOutput())


    image_data["R"][image_type] = pv.create_grid(
        image_data["R"][image_type],
        dimensions=[d//resample_factor for d in image_data["R"][image_type].dimensions]
    ).sample(image_data["R"][image_type], categorical=True, progress_bar=True)
    
    arr = image_data["R"][image_type]["AIMData"].reshape(image_data["R"][image_type].dimensions, order="F")
    for d in range(3):
        for i in [0,-1]:
            st = [slice(None), slice(None), slice(None)]
            st[d] = i
            st = tuple(st)
            arr[st] = 0
    image_data["R"][image_type]["AIMData"] = arr.flatten(order="F")

Resampling array Data from a Passed Mesh onto Mesh: 100%|██████████████████████████████████████████████████████████████████████████████████████[00:00<00:00]
Resampling array Data from a Passed Mesh onto Mesh: 100%|██████████████████████████████████████████████████████████████████████████████████████[00:00<00:00]


In [28]:
progress_bar = True

for img in image_data.values():
    img["CORT contour"] = img["CORT"].contour([126], progress_bar=progress_bar)

Computing Contour: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████[00:01<00:00]
Computing Contour: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████[00:01<00:00]
Computing Contour: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████[00:00<00:00]


In [ ]:
pl = pv.Plotter(
    notebook=False, 
    lighting='three lights', 
    window_size=(1200,600),
    line_smoothing=True,
    polygon_smoothing=True,
    #theme=pv.themes.DocumentTheme(),
)

for bone in ["F", "T", "R"]:
    pl.add_mesh(
        image_data[bone]["CORT contour"].rotate_z(
            90 if bone=="R" else 0,
            (
                image_data[bone]["CORT contour"].points.min(axis=0)
                + (
                    image_data[bone]["CORT contour"].points.max(axis=0)
                    - image_data[bone]["CORT contour"].points.min(axis=0)
                )/2
            )
        ).translate(
            [
                65 if bone=="R" else 0,
                -15 if bone=="R" else 0,
                20 if bone=="R" else 0
            ]
        ),
        color="lightgray"
    )
        
pl.show()